# Investigating the Cybersecurity Intrusion Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

# Comment created in john-feature branch

In [2]:
df = pd.read_csv('../data/cybersecurity_intrusion_data.csv')
df.head(3)

session_id  network_packet_size protocol_type  login_attempts  \
0  SID_00001                  599           TCP               4   
1  SID_00002                  472           TCP               3   
2  SID_00003                  629           TCP               3   

   session_duration encryption_used  ip_reputation_score  failed_logins  \
0        492.983263             DES             0.606818              1   
1       1557.996461             DES             0.301569              0   
2         75.044262             DES             0.739164              2   

  browser_type  unusual_time_access  attack_detected  
0         Edge                    0                1  
1      Firefox                    0                0  
2       Chrome                    0                1

### Examine the dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9537 entries, 0 to 9536
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   session_id           9537 non-null   object 
 1   network_packet_size  9537 non-null   int64  
 2   protocol_type        9537 non-null   object 
 3   login_attempts       9537 non-null   int64  
 4   session_duration     9537 non-null   float64
 5   encryption_used      7571 non-null   object 
 6   ip_reputation_score  9537 non-null   float64
 7   failed_logins        9537 non-null   int64  
 8   browser_type         9537 non-null   object 
 9   unusual_time_access  9537 non-null   int64  
 10  attack_detected      9537 non-null   int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 819.7+ KB


### Check the shape of the dataframe

In [4]:
df.shape

(9537, 11)

### Drop the session_id feature

In [5]:
# Drop the 'session_id' column
df.drop('session_id', axis=1, inplace=True)
df.columns

Index(['network_packet_size', 'protocol_type', 'login_attempts',
       'session_duration', 'encryption_used', 'ip_reputation_score',
       'failed_logins', 'browser_type', 'unusual_time_access',
       'attack_detected'],
      dtype='object')

### Check for null values

The feature 'encryption_used' is the only feature that suffers from null values.


In [6]:
df.isnull().sum()

network_packet_size       0
protocol_type             0
login_attempts            0
session_duration          0
encryption_used        1966
ip_reputation_score       0
failed_logins             0
browser_type              0
unusual_time_access       0
attack_detected           0
dtype: int64

In [7]:
df['encryption_used'].value_counts()

encryption_used
AES    4706
DES    2865
Name: count, dtype: int64

### Let's replace null values in the encryption_used feature with 'unknown'

In [8]:
# Replace null values in the 'encryption_used' column with a new value 'unknown'
df.fillna({'encryption_used': 'unknown'}, inplace=True)

# And check the result
df.isnull().sum()

network_packet_size    0
protocol_type          0
login_attempts         0
session_duration       0
encryption_used        0
ip_reputation_score    0
failed_logins          0
browser_type           0
unusual_time_access    0
attack_detected        0
dtype: int64

### 

### Let's check the value counts of the categorical variables

In [9]:
# Check the value counts of the categorical variables
for column in df.select_dtypes(include='object').columns:
    print(f'{column}:\n{df[column].value_counts()}\n')

protocol_type:
protocol_type
TCP     6624
UDP     2406
ICMP     507
Name: count, dtype: int64

encryption_used:
encryption_used
AES        4706
DES        2865
unknown    1966
Name: count, dtype: int64

browser_type:
browser_type
Chrome     5137
Firefox    1944
Edge       1469
Unknown     502
Safari      485
Name: count, dtype: int64



In [10]:
# Use one-hot encoding to convert the categorical variables into numerical ones
# Use the sklearn library to do this
from sklearn.preprocessing import OneHotEncoder

# Create the OneHotEncoder object
encoder = OneHotEncoder(drop='first', sparse_output=False)

categorical_columns = df.select_dtypes(include='object').columns.to_list()

one_hot_encoded = encoder.fit_transform(df[categorical_columns])

one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

df_encoded = pd.concat([df, one_hot_encoded_df], axis=1)
df_encoded.drop(categorical_columns, axis=1, inplace=True)

categorical_columns

['protocol_type', 'encryption_used', 'browser_type']

In [11]:
# Create a new dataframe with only the numeric columns
df_numeric = df_encoded

# Show the correlation matrix
df_numeric_corr = df_numeric.corr()
df_numeric_corr


network_packet_size  login_attempts  \
network_packet_size                 1.000000       -0.001890   
login_attempts                     -0.001890        1.000000   
session_duration                    0.021650        0.006392   
ip_reputation_score                 0.002320       -0.002618   
failed_logins                      -0.011676       -0.013507   
unusual_time_access                -0.001255        0.007349   
attack_detected                    -0.006798        0.277320   
protocol_type_TCP                  -0.017863       -0.023837   
protocol_type_UDP                   0.016565        0.026665   
encryption_used_DES                 0.009977        0.002339   
encryption_used_unknown             0.008647        0.000254   
browser_type_Edge                  -0.005308        0.000720   
browser_type_Firefox                0.017610        0.002735   
browser_type_Safari                 0.013130       -0.007188   
browser_type_Unknown                0.002234       -0.004092   

                         session_duration  ip_reputation_score  failed_logins  \
network_packet_size              0.021650             0.002320      -0.011676   
login_attempts                   0.006392            -0.002618      -0.013507   
session_duration                 1.000000            -0.005077       0.019375   
ip_reputation_score             -0.005077             1.000000       0.015613   
failed_logins                    0.019375             0.015613       1.000000   
unusual_time_access              0.012930            -0.003146       0.006131   
attack_detected                  0.041602             0.211540       0.363726   
protocol_type_TCP                0.010986            -0.022273       0.010408   
protocol_type_UDP               -0.007982             0.030000      -0.009051   
encryption_used_DES             -0.008657            -0.011392       0.005216   
encryption_used_unknown          0.008510             0.006522      -0.010514   
browser_type_Edge               -0.012287             0.002337       0.002919   
browser_type_Firefox            -0.001914             0.001258      -0.002404   
browser_type_Safari              0.010599             0.017507       0.004561   
browser_type_Unknown             0.003758            -0.007540      -0.003144   

                         unusual_time_access  attack_detected  \
network_packet_size                -0.001255        -0.006798   
login_attempts                      0.007349         0.277320   
session_duration                    0.012930         0.041602   
ip_reputation_score                -0.003146         0.211540   
failed_logins                       0.006131         0.363726   
unusual_time_access                 1.000000         0.008652   
attack_detected                     0.008652         1.000000   
protocol_type_TCP                  -0.005240         0.000643   
protocol_type_UDP                   0.004219         0.007903   
encryption_used_DES                -0.001656         0.008306   
encryption_used_unknown            -0.004201         0.016161   
browser_type_Edge                   0.005479        -0.008057   
browser_type_Firefox                0.002561        -0.010556   
browser_type_Safari                 0.000372        -0.013289   
browser_type_Unknown               -0.014824         0.134630   

                         protocol_type_TCP  protocol_type_UDP  \
network_packet_size              -0.017863           0.016565   
login_attempts                   -0.023837           0.026665   
session_duration                  0.010986          -0.007982   
ip_reputation_score              -0.022273           0.030000   
failed_logins                     0.010408          -0.009051   
unusual_time_access              -0.005240           0.004219   
attack_detected                   0.000643           0.007903   
protocol_type_TCP                 1.000000          -0.875916   
protocol_type_UDP                -0.875916           1.000000   
encryption_u

### Visualize the correlation matrix as a heatmap

In [12]:
# Create a heatmap of the correlation matrix, with values correct to 2 decimal places
plt.figure(figsize=(12, 12))
sns.heatmap(df_numeric_corr, annot=True, cmap='coolwarm', fmt='.2f')

# Save the heatmap image to file
plt.savefig('../images/heatmap.png')

### Create a pipeline to preprocess the data and train a model

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer

def pipeline_logistic_regression():
    """
    Create a pipeline for logistic regression.

    Returns:
        pipeline (Pipeline): A pipeline object that consists of feature scaling, feature selection, and logistic regression model.
    """
    
    categorical_features = df.select_dtypes(include='object').columns
    print(categorical_features)

    numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
    numerical_features = numerical_features.drop('attack_detected')
    print(numerical_features)
    
    # Create transformers
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    numerical_transformer = StandardScaler()
    
    # Combine transformations using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )
    
        
    pipeline = Pipeline(
        [
            ("preprocessor", preprocessor),
            # ("feature_selection", SelectFromModel(LogisticRegression(random_state=101))),
            ("model", LogisticRegression(random_state=101)),
        ]
    )
    
    

    return pipeline




### Split the data into train and test sets, and fit the pipeline

In [14]:
# split the data into training and testing sets
from sklearn.model_selection import train_test_split

X = df.drop('attack_detected', axis=1)

y = df['attack_detected']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

pipeline = pipeline_logistic_regression()
pipeline.fit(X_train, y_train)

Index(['protocol_type', 'encryption_used', 'browser_type'], dtype='object')
Index(['network_packet_size', 'login_attempts', 'session_duration',
       'ip_reputation_score', 'failed_logins', 'unusual_time_access'],
      dtype='object')


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['network_packet_size', 'login_attempts', 'session_duration',
       'ip_reputation_score', 'failed_logins', 'unusual_time_access'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['protocol_type', 'encryption_used', 'browser_type'], dtype='object'))])),
                ('model', LogisticRegression(random_state=101))])

Create a function to get the coefficients from the model

In [15]:
def logistic_regression_coef(model, columns):
    """
    Prints the coefficients of a logistic regression model.

    Parameters:
    - model: The trained logistic regression model.
    - columns: The column names corresponding to the coefficients.

    Returns:
    None
    """
    coeff_df = pd.DataFrame(
        model.coef_, index=["Coefficient"], columns=columns
    ).T.sort_values(["Coefficient"], key=abs, ascending=False)
    print(coeff_df)

In [16]:
logistic_regression_coef(
    model=pipeline["model"],
    columns=pipeline["preprocessor"].get_feature_names_out(),
)


                              Coefficient
cat__browser_type_Unknown        1.434387
num__failed_logins               1.006663
num__login_attempts              0.785050
num__ip_reputation_score         0.572452
cat__browser_type_Safari        -0.517531
cat__browser_type_Edge          -0.321258
cat__browser_type_Firefox       -0.315247
cat__browser_type_Chrome        -0.313979
cat__protocol_type_ICMP         -0.109423
cat__encryption_used_AES        -0.109289
num__session_duration            0.094870
cat__protocol_type_TCP           0.067447
cat__encryption_used_unknown     0.064580
num__unusual_time_access         0.033093
num__network_packet_size        -0.026275
cat__encryption_used_DES         0.011080
cat__protocol_type_UDP           0.008348


### Generate the classification report and view the confusion matrix

In [17]:
pipeline["preprocessor"].get_feature_names_out()

array(['num__network_packet_size', 'num__login_attempts',
       'num__session_duration', 'num__ip_reputation_score',
       'num__failed_logins', 'num__unusual_time_access',
       'cat__protocol_type_ICMP', 'cat__protocol_type_TCP',
       'cat__protocol_type_UDP', 'cat__encryption_used_AES',
       'cat__encryption_used_DES', 'cat__encryption_used_unknown',
       'cat__browser_type_Chrome', 'cat__browser_type_Edge',
       'cat__browser_type_Firefox', 'cat__browser_type_Safari',
       'cat__browser_type_Unknown'], dtype=object)

### Try predicting a new datapoint

In [18]:
X_new = pd.DataFrame({
    'network_packet_size': [599],
    'protocol_type': ['TCP'],
    'login_attempts': [3],
    'session_duration': [1557],
    'encryption_used': ['DES'],
    'ip_reputation_score': [0.301568],
    'failed_logins': [0],
    'browser_type': ['Firefox'],
    'unusual_time_access': [0],
})

categorical_features = df.select_dtypes(include='object').columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('attack_detected')

expected_features = list(numerical_features) + list(categorical_features)
X_new = X_new.reindex(columns=expected_features, fill_value=np.nan)  # Fill missing columns with NaN

prediction = pipeline.predict(X_new)
print(prediction)

probabilites = pipeline.predict_proba(X_new)
print("Probabilities of each class:", probabilites)

[0]
Probabilities of each class: [[0.8988224 0.1011776]]


In [19]:
df['unusual_time_access'].value_counts()
df.dtypes

network_packet_size      int64
protocol_type           object
login_attempts           int64
session_duration       float64
encryption_used         object
ip_reputation_score    float64
failed_logins            int64
browser_type            object
unusual_time_access      int64
attack_detected          int64
dtype: object

In [20]:
# loads confusion_matrix and classification_report from sklearn
from sklearn.metrics import classification_report, confusion_matrix


def confusion_matrix_and_report(X, y, pipeline, label_map):
    """
    Gets features, target, pipeline, and how the levels from your target are labelled (named).
    In this case, 0 (Malignant) and 1 (Benign), so you parse a list ['Malignant' , 'Benign'].

    Args:
        X (array-like): The input features.
        y (array-like): The target values.
        pipeline (object): The trained pipeline model.
        label_map (list): The list of labels for the target values.

    Returns:
        None

    This function performs the following steps:
    - Predicts the target values based on the input features using the provided pipeline.
    - Computes and displays the confusion matrix, which compares the predicted values with the actual values.
      The predicted values are shown as rows, and the actual values are shown as columns in the matrix.
    - Displays the classification report, which provides metrics such as precision, recall, and F1-score.

    """

    prediction = pipeline.predict(X)

    print("---  Confusion Matrix  ---")
    print(
        pd.DataFrame(
            confusion_matrix(y_true=prediction, y_pred=y),
            columns=[["Actual " + sub for sub in label_map]],
            index=[["Prediction " + sub for sub in label_map]],
        )
    )
    print("\n")

    print("---  Classification Report  ---")
    print(classification_report(y, prediction, target_names=label_map), "\n")


def clf_performance(X_train, y_train, X_test, y_test, pipeline, label_map):
    """
    Calculates and displays the performance metrics of a classification model.

    Parameters:
    - X_train (array-like): The feature matrix of the training set.
    - y_train (array-like): The target labels of the training set.
    - X_test (array-like): The feature matrix of the test set.
    - y_test (array-like): The target labels of the test set.
    - pipeline (object): The trained classification pipeline.
    - label_map (list): A list containing the labels for the target variable.

    Returns:
    None

    This function calculates and displays the confusion matrix and classification report
    for both the training set and the test set. The confusion matrix provides a summary
    of the model's performance by showing the number of true positives, true negatives,
    false positives, and false negatives. The classification report provides additional
    performance metrics such as precision, recall, and F1-score.

    Example usage:
    clf_performance(X_train, y_train, X_test, y_test, pipeline, ['Malignant', 'Benign'])
    """
    print("#### Train Set #### \n")
    confusion_matrix_and_report(X_train, y_train, pipeline, label_map)
    
    print("#### Test Set ####\n")
    confusion_matrix_and_report(X_test, y_test, pipeline, label_map)

In [21]:
clf_performance(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    pipeline=pipeline,
    label_map=["Attack", "No Attack"],
)

#### Train Set #### 

---  Confusion Matrix  ---
                     Actual Attack Actual No Attack
Prediction Attack             3370             1114
Prediction No Attack           847             2298


---  Classification Report  ---
              precision    recall  f1-score   support

      Attack       0.75      0.80      0.77      4217
   No Attack       0.73      0.67      0.70      3412

    accuracy                           0.74      7629
   macro avg       0.74      0.74      0.74      7629
weighted avg       0.74      0.74      0.74      7629
 

#### Test Set ####

---  Confusion Matrix  ---
                     Actual Attack Actual No Attack
Prediction Attack              838              286
Prediction No Attack           218              566


---  Classification Report  ---
              precision    recall  f1-score   support

      Attack       0.75      0.79      0.77      1056
   No Attack       0.72      0.66      0.69       852

    accuracy                   